<a href="https://colab.research.google.com/github/smozley/austinAIallianceintensive/blob/main/multi_agent_orchestration_tutorial_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This tutorial guides you through a simple multi-agent network system from scratch.

### **Step 1: Setup and Installation**

### Setting up Ollama (in the terminal window)

1. Install the dependencies

```
sudo apt update
sudo apt install -y pciutils
curl -fsSL https://ollama.com/install.sh | sh
```

2. Run the ollama server in the background

```
ollama serve &
```

3. Pull the required models

```
ollama pull granite3.3:latest
```

4. Run the models in the background

```
ollama run granite3.3:latest &
```

### Installing the necessary libraries

In [ ]:
!pip install langgraph langchain langchain_openai

### **Step 2: Initialize Model**

We will configure our LLMs using the `ChatOpenAI` wrapper.

In [ ]:
from typing import TypedDict, Annotated, Literal
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages

# This connects to your local Ollama instance.
# Make sure the model name matches what you have pulled.
# You can use "granite3.3:latest", "mistral:latest", etc.
llm = ChatOpenAI(
    base_url='http://localhost:11434/v1',
    api_key='ollama',  # required but can be any string
    model='granite3.3:latest',
    temperature=0.9, # A little creativity for joke writing
)

print("✅ LLM Initialized")

### **Step 3: Define Agent State and Nodes**

In [ ]:
# This is the shared "story document" for our agents.
class StoryState(TypedDict):
    # The list of messages that make up the story so far.
    messages: Annotated[list[BaseMessage], add_messages]
    # A turn counter to control the length of the story.
    turn: int
    # The maximum number of turns for the story.
    max_turns: int


# These are our two specialist writers.

def world_builder_node(state: StoryState):
    """
    The World Builder agent. It focuses on creating the setting and atmosphere.
    """
    print(f" Turn {state['turn'] + 1}: World Builder is writing... ")

    prompt = ChatPromptTemplate.from_messages([
        ("system",
         "You are a master world-builder. Your role is to describe the setting, characters, and atmosphere. "
         "Read the story so far, and add a paragraph that expands on the description of the world. "
         "Do NOT advance the plot or make characters speak. Focus only on descriptive details."),
        ("placeholder", "{messages}")
    ])

    chain = prompt | llm | StrOutputParser()
    story_segment = chain.invoke({"messages": state['messages']})

    print(story_segment)
    return {
        "messages": [HumanMessage(content=story_segment, name="World Builder")],
        "turn": state['turn'] + 1
    }


def plot_weaver_node(state: StoryState):
    """
    The Plot Weaver agent. It focuses on action, dialogue, and advancing the story.
    """
    print(f" Turn {state['turn'] + 1}: Plot Weaver is writing... ")

    prompt = ChatPromptTemplate.from_messages([
        ("system",
         "You are a master storyteller. Your role is to advance the plot. "
         "Read the story so far, including the world-building, and add a paragraph that introduces an action, a decision, or a line of dialogue. "
         "Do NOT add descriptive details about the setting. Focus only on moving the story forward."),
        ("placeholder", "{messages}")
    ])

    chain = prompt | llm | StrOutputParser()
    story_segment = chain.invoke({"messages": state['messages']})

    print(story_segment)
    return {
        "messages": [HumanMessage(content=story_segment, name="Plot Weaver")],
        "turn": state['turn'] + 1
    }



def router_node(state: StoryState) -> Literal["world_builder", "plot_weaver", END]:
    """
    This function determines which agent should write next.
    """
    # If we have reached the maximum number of turns, the story ends.
    if state['turn'] >= state['max_turns']:
        print(" Story Complete ")
        return END

    # Check the name of the last speaker to decide who goes next.
    last_speaker = state["messages"][-1].name
    if last_speaker == "World Builder":
        return "plot_weaver"
    else:
        return "world_builder"

### **Step 4: Build the Multi-Agent Network Graph**

Now we wire the nodes together.

In [ ]:
# This graph implements the dynamic, peer-to-peer collaboration.

workflow = StateGraph(StoryState)

# Add the two specialist agent nodes.
workflow.add_node("world_builder", world_builder_node)
workflow.add_node("plot_weaver", plot_weaver_node)

# The story always starts with the World Builder to set the scene.
workflow.set_entry_point("world_builder")

# Define the conditional routing.
# After the World Builder, we call the router to decide the next step.
workflow.add_conditional_edges(
    "world_builder",
    router_node,
    {"plot_weaver": "plot_weaver", END: END}
)
# After the Plot Weaver, we also call the router.
workflow.add_conditional_edges(
    "plot_weaver",
    router_node,
    {"world_builder": "world_builder", END: END}
)

# Compile the graph.
graph = workflow.compile()
print("✅ Creative Writing Duo Graph Compiled")

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

### **Step 5: Run the Graph**

In [ ]:
if __name__ == "__main__":
    story_premise = "A lone astronaut discovers a mysterious, silent alien monolith on Mars."
    max_turns = 6 # The story will have 6 paragraphs (3 from each agent).

    print(f"\n Beginning a new story: '{story_premise}'\n")

    # Using .invoke() is simpler for getting the final result after the full run.
    # It waits for the graph to complete and returns the final, merged state.
    final_state = graph.invoke(
        {
            "messages": [HumanMessage(content=story_premise, name="Story Premise")],
            "turn": 0,
            "max_turns": max_turns
        },
        # Set a high recursion limit to allow for many turns in the conversation.
        {"recursion_limit": 100}
    )

    print("\n\n" + "#"*60)
    print("           THE COMPLETE STORY")
    print("#"*60 + "\n")

    full_story = "\n\n".join([msg.content for msg in final_state['messages']])
    print(full_story)